# Classifieurs - Spotify Best Songs from 2000 to 2023
-> Prédiction du genre des musiques populaires de 2000 à 2023

- SCHERRER Arthur
- CHIRON Yoann
- BOQUAIN Mathis

### Import des différentes librairies

In [152]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics

### Chargement des données

In [148]:
data = pd.read_csv("spotify_songs.csv", sep=';')

### Affichage

In [149]:
df = pd.DataFrame(data)
df.head()


,title,artist,top genre,year,bpm,energy,danceability,valence,duration,acousticness,speechiness
0,Flowers,Miley Cyrus,pop,2023,118,68,71,65,200,6,7
1,Cupid - Twin Ver.,FIFTY FIFTY,k-pop girl group,2023,120,59,78,73,174,44,3
2,BESO,ROSALÍA,pop,2023,95,64,77,53,195,74,14
3,Boy's a liar Pt. 2,PinkPantheress,bronx drill,2023,133,81,70,86,131,25,5
4,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,rap,2022,98,62,72,17,222,42,5


### Discrétisation des données

Certaines features ont été supprimées où ne sont pas utilisées car peu de pertinences face au sujet étudié :
- dB
- liveness
- popularity : tous les morceaux sont populaires

#### Feature à discrétiser

- genre
- bpm
- danceability
- energy
- valence
- acousticness
- speechiness
- year 
- duration

### Discrétisation du genre

In [150]:
df_sorted = df.sort_values(by="top genre")

df['top genre'] = df['top genre'].str.replace('.*pop.*', 'pop', regex=True)
df['top genre'] = df['top genre'].str.replace('.*hip hop.*', 'rap', regex=True)
df['top genre'] = df['top genre'].str.replace('.*hip-hop.*', 'rap', regex=True)
df['top genre'] = df['top genre'].str.replace('.*rap.*', 'rap', regex=True)
df['top genre'] = df['top genre'].str.replace('.*metal.*', 'metal', regex=True)
df['top genre'] = df['top genre'].str.replace('.*r&b.*', 'r&b', regex=True)
df['top genre'] = df['top genre'].str.replace('.*edm.*', 'techno', regex=True)
df['top genre'] = df['top genre'].str.replace('.*electro.*', 'techno', regex=True)
df['top genre'] = df['top genre'].str.replace('.*electro dance music.*', 'techno', regex=True)
df['top genre'] = df['top genre'].str.replace('.*indie.*', 'indian', regex=True)
df['top genre'] = df['top genre'].str.replace('.*rock.*', 'rock', regex=True)
df['top genre'] = df['top genre'].str.replace('.*house.*', 'techno', regex=True)
df['top genre'] = df['top genre'].str.replace('.*jazz.*', 'jazz', regex=True)
df['top genre'] = df['top genre'].str.replace('.*mellow.*', 'rock', regex=True)
df['top genre'] = df['top genre'].str.replace('.*soul.*', 'soul', regex=True)

genres_to_keep = ['pop', 'rap', 'metal', 'r&b', 'techno', 'rock']
mask = df['top genre'].isin(genres_to_keep)
df = df[mask]

df_count = df['top genre'].value_counts()
df.head()

,title,artist,top genre,year,bpm,energy,danceability,valence,duration,acousticness,speechiness
0,Flowers,Miley Cyrus,pop,2023,118,68,71,65,200,6,7
1,Cupid - Twin Ver.,FIFTY FIFTY,pop,2023,120,59,78,73,174,44,3
2,BESO,ROSALÍA,pop,2023,95,64,77,53,195,74,14
4,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,rap,2022,98,62,72,17,222,42,5
8,Anti-Hero,Taylor Swift,pop,2022,97,64,64,53,201,13,5


### Méthode de discrétisation

In [159]:
def discretize_column(df, column_name, num_intervals, labels=None):
    """
    Discrétise une colonne de données dans un DataFrame en fonction d'un nombre d'intervalles spécifié.

    Args:
    - df : DataFrame contenant les données
    - column_name : Nom de la colonne à discrétiser
    - num_intervals : Nombre d'intervalles souhaité
    - labels : Liste des labels pour les catégories discrétisées

    Returns:
    - Un DataFrame avec la colonne initiale remplacée par les données discrétisées
    """

    # Calcul des quantiles en fonction du nombre d'intervalles
    quantiles = [i / num_intervals for i in range(num_intervals)]

    # Calcul des valeurs des quantiles
    quantiles_values = df[column_name].quantile(quantiles)

    # Définition des intervalles
    intervals = [df[column_name].min()] + list(quantiles_values.unique()) + [df[column_name].max()]

    # Définition des labels si spécifiés, sinon utiliser les intervalles comme labels
    if labels is None:
        labels = [f"Interval {i+1}" for i in range(len(quantiles) + 1)]

    # Discrétisation de la colonne en supprimant les bords d'intervalles en double
    df[column_name] = pd.cut(df[column_name], bins=intervals, labels=labels, include_lowest=True, duplicates='drop')

    return df

df = discretize_column(df, 'bpm', num_intervals=4, labels=['Low bpm', 'Medium bpm', 'High bpm', 'Very High bpm'])
df = discretize_column(df, 'danceability', num_intervals=4, labels=['Low Danceability', 'Moderate Danceability', 'High Danceability', 'Very High Danceability'])
df = discretize_column(df, 'energy', num_intervals=4, labels=['Low Energy', 'Moderate Energy', 'High Energy', 'Very High Energy'])
df = discretize_column(df, 'valence', num_intervals=3, labels=['negative', 'neutral', 'positive'])
df = discretize_column(df, 'acousticness', num_intervals=4, labels= ['Low Acousticness', 'Moderate Acousticness', 'High Acousticness', 'Very High Acousticness'])
df = discretize_column(df, 'speechiness', num_intervals=4, labels=['Low Speechiness', 'Moderate Speechiness', 'High Speechiness', 'Very High Speechiness'])
df = discretize_column(df, 'year', num_intervals=3, labels=["2000's","2010's", "2020's"])
df = discretize_column(df, 'duration', num_intervals=3, labels=['short', 'medium', 'long'])

df.head()

AssertionError: 

### Séparation du dataset 

In [161]:
x = df[['bpm','danceability','energy','valence','acousticness','speechiness','year','duration']]
y = df['top genre']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=101)

df

,title,artist,top genre,year,bpm,energy,danceability,valence,duration,acousticness,speechiness
0,Flowers,Miley Cyrus,pop,2020's,Medium bpm,Moderate Energy,High Danceability,neutral,short,Moderate Acousticness,High Speechiness
1,Cupid - Twin Ver.,FIFTY FIFTY,pop,2020's,Medium bpm,Low Energy,Very High Danceability,positive,short,Very High Acousticness,Low Speechiness
2,BESO,ROSALÍA,pop,2020's,Low bpm,Moderate Energy,High Danceability,neutral,short,Very High Acousticness,Very High Speechiness
4,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,rap,2020's,Low bpm,Moderate Energy,High Danceability,negative,medium,Very High Acousticness,Moderate Speechiness
8,Anti-Hero,Taylor Swift,pop,2020's,Low bpm,Moderate Energy,Moderate Danceability,neutral,short,High Acousticness,Moderate Speechiness
...,...,...,...,...,...,...,...,...,...,...,...
2374,NOT AFRAID,Kodiro,rap,2020's,Low bpm,Low Energy,High Danceability,negative,short,Moderate Acousticness,Low Speechiness
2375,Pop Ya Collar - Radio Edit,Usher,rap,2020's,Medium bpm,High Energy,Very High Danceability,positive,medium,High Acousticness,High Speechiness
2379,Southbound,Beach Blvd,rock,2020's,Very High bpm,Very High Energy,Low Danceability,neutral,medium,High Acousticness,Low Speechiness
2382,Scared of the Dark,Everything Brighter,pop,2020's,Medium bpm,High Energy,Moderate Danceability,neutral,short,Low Acousticness,Moderate Speechiness


# Supervisé

### ZeroR

In [174]:
def learn_zeror(x_train, y_train):
    unique_labels, counts = np.unique(y_train, return_counts=True)

    # Trouvez l'indice de l'étiquette avec le nombre le plus élevé
    index_max_count = np.argmax(counts)
    
    # L'étiquette avec le nombre le plus élevé est à l'indice index_max_count
    most_common_label = unique_labels[index_max_count]
    
    return most_common_label 

def class_zeror(model, x_test):
    return [model]* len(x_test)

### OneR

### Naïve Bayes

### KPPV

# Non Supervisé

### K-means

### Méthodes d'évaluation

In [162]:
def evaluate(Class_true, Class_pred):
    acc = accuracy(Class_true, Class_pred)
    prec = precision(Class_true, Class_pred)
    rap = rappel(Class_true, Class_pred)
    return acc, prec, rap

def accuracy(Class_true, Class_pred):
    correct = sum(1 for true, pred in zip(Class_true, Class_pred) if true == pred)
    total = len(Class_pred)
    return "accuracy =  " + str(correct / total * 100) + "%"

def precision(Class_true, Class_pred):
    unique_values = np.unique(Class_true)
    n = 0
    for value in unique_values:
        correct = sum(1 for true, pred in zip(Class_true, Class_pred) if true == pred == value)
        total = sum(1 for pred in Class_pred if pred == value)
        if total != 0 :
            n += correct / total
    return "précision =  " + str(n / len(unique_values) * 100) + "%"

def rappel(Class_true, Class_pred):
    unique_values = np.unique(Class_true)
    n = 0
    for value in unique_values:
        correct = sum(1 for true, pred in zip(Class_true, Class_pred) if true == pred == value)
        total = sum(1 for true in Class_true if true == value)
        if total != 0 :
            n += correct / total
    return "rappel =  " + str(n / len(unique_values) * 100) + "%"

In [ ]:
def stats(test):
     
     print(metrics.classification_report(test.target, 
                                         test.hyp, 
                                         target_names=test.target_name.unique(), 
                                         zero_division=1))

In [176]:
learn_zeror(x_train, y_train)

pred2 = class_zeror(learn_zeror(x_train, y_train), x_test)

evaluate(y_test, pred2)

('accuracy =  54.947916666666664%',
 'précision =  9.15798611111111%',
 'rappel =  16.666666666666664%')